# Day 4 - Lab 2: Generating a CI/CD Pipeline (Solution)

**Objective:** Use an LLM to generate all necessary configuration files to create an automated Continuous Integration (CI) pipeline for the FastAPI application using Docker and GitHub Actions.

**Introduction:**
This solution notebook provides the complete prompts for generating the `requirements.txt`, `Dockerfile`, and GitHub Actions workflow files. It demonstrates how to prompt for specific, structured configuration-as-code artifacts.

For definitions of key terms used in this lab, please refer to the [GLOSSARY.md](../../GLOSSARY.md).

## Step 1: Setup

In [1]:
import sys
import os

# Add the project's root directory to the Python path to ensure 'utils' can be imported.
try:
    project_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
except IndexError:
    project_root = os.path.abspath(os.path.join(os.getcwd()))

if project_root not in sys.path:
    sys.path.insert(0, project_root)

from utils import setup_llm_client, get_completion, save_artifact, load_artifact, clean_llm_output

client, model_name, api_provider = setup_llm_client(model_name="gpt-4o")

# Load the application code from Day 3 to provide context
app_code = load_artifact("app/main.py")
if not app_code:
    print("Warning: Could not load app/main.py. Lab may not function correctly.")

✅ LLM Client configured: Using 'openai' with model 'gpt-4o'


## Step 2: The Challenges - Solutions

### Challenge 1 (Foundational): Generating a `requirements.txt`

**Explanation:**
This prompt is an analysis task. We provide the application's source code as context and ask the LLM to parse the import statements to identify external libraries. This is much faster and less error-prone than manually creating the file.

In [5]:
requirements_prompt = f"""You are a Python dependency analysis tool. Analyze the following Python code and generate a `requirements.txt` file listing all the external libraries imported.

Include versions for key libraries like fastapi, uvicorn, sqlalchemy, and pydantic. Also include pytest for running tests.

--- PYTHON CODE ---
{app_code}
--- END CODE ---

Output only the contents of the requirements.txt file.
"""

print("--- Generating requirements.txt ---")
if app_code:
    requirements_content = get_completion(requirements_prompt, client, model_name, api_provider)
    cleaned_reqs = clean_llm_output(requirements_content, language='text')
    print(cleaned_reqs)
    save_artifact(cleaned_reqs, "requirements.txt")
else:
    print("Skipping requirements generation because app code is missing.")

--- Generating requirements.txt ---
plaintext
fastapi==0.95.2
uvicorn==0.22.0
sqlalchemy==2.0.21
pydantic==1.10.4
pytest==7.4.2
✅ Successfully saved artifact to: requirements.txt


### Challenge 2 (Intermediate): Generating a `Dockerfile`

**Explanation:**
This prompt asks for a specific, best-practice `Dockerfile`. By explicitly requesting a "multi-stage" build, a "slim" base image, and a non-root user, we guide the LLM to generate a configuration that is both efficient (smaller final image size) and secure, saving the developer from having to remember these important details.

In [7]:
dockerfile_prompt = """You are a DevOps expert specializing in containerization. Generate a best-practice, multi-stage `Dockerfile` for a production Python FastAPI application.

The Dockerfile must:
1.  Use `python:3.11-slim` as the base image.
2.  The first stage should install dependencies from `requirements.txt`.
3.  The final stage should copy the application code and the installed dependencies from the first stage.
4.  Expose port 8000.
5.  The final `CMD` should run the application using `uvicorn`, binding to host 0.0.0.0.

Output only the raw Dockerfile content.
"""

print("--- Generating Dockerfile ---")
dockerfile_content = get_completion(dockerfile_prompt, client, model_name, api_provider)
cleaned_dockerfile = clean_llm_output(dockerfile_content, language='dockerfile')
print(cleaned_dockerfile)

if cleaned_dockerfile:
    save_artifact(cleaned_dockerfile, "Dockerfile")

--- Generating Dockerfile ---
Dockerfile
# Use the official Python image as the base for the first stage
FROM python:3.11-slim as builder

# Set the working directory
WORKDIR /app

# Copy the requirements file into the container
COPY requirements.txt .

# Install dependencies
RUN pip install --no-cache-dir -r requirements.txt

# Use the official Python image as the base for the final stage
FROM python:3.11-slim

# Set the working directory
WORKDIR /app

# Copy the application code into the container
COPY . .

# Copy the installed dependencies from the builder stage
COPY --from=builder /usr/local/lib/python3.11/site-packages /usr/local/lib/python3.11/site-packages
COPY --from=builder /usr/local/bin /usr/local/bin

# Expose the application port
EXPOSE 8000

# Run the application
CMD ["uvicorn", "main:app", "--host", "0.0.0.0", "--port", "8000"]
✅ Successfully saved artifact to: Dockerfile


### Challenge 3 (Advanced): Generating the GitHub Actions Workflow

**Explanation:**
This prompt generates a complete CI workflow in YAML format. Defining our CI pipeline in a YAML file (`ci.yml`) is a core DevOps practice known as 'Configuration as Code.' It makes our build and test process version-controlled, repeatable, and easy to review, just like our application code. We are very specific about the required structure: the `on` trigger, the `jobs` definition, and the `steps` within the job. This level of detail ensures the LLM generates a syntactically correct and logically complete workflow file that is ready to be committed to the repository.

In [10]:
ci_workflow_prompt = """You are a CI/CD specialist. Generate a complete GitHub Actions workflow file named `ci.yml` for a Python FastAPI project.

The workflow must:
- Be named 'Build and Test'.
- Trigger on every `push` to the `main` branch.
- Define one job named `build-and-test` that runs on `ubuntu-latest`.
- The job must have the following sequential steps:
  1. `actions/checkout@v4` to check out the repository code.
  2. `actions/setup-python@v5` to set up Python 3.11.
  3. A step to install dependencies using pip from `requirements.txt`.
  4. A step to run the test suite using `pytest`.

Output only the raw YAML content for the file.
"""

print("--- Generating GitHub Actions Workflow ---")
ci_workflow_content = get_completion(ci_workflow_prompt, client, model_name, api_provider)
cleaned_yaml = clean_llm_output(ci_workflow_content, language='yaml')
print(cleaned_yaml)

if cleaned_yaml:
    save_artifact(cleaned_yaml, ".github/workflows/ci.yml")

--- Generating GitHub Actions Workflow ---
name: Build and Test

on:
  push:
    branches:
      - main

jobs:
  build-and-test:
    runs-on: ubuntu-latest

    steps:
      - name: Check out repository code
        uses: actions/checkout@v4

      - name: Set up Python 3.11
        uses: actions/setup-python@v5
        with:
          python-version: '3.11'

      - name: Install dependencies
        run: pip install -r requirements.txt

      - name: Run test suite
        run: pytest
✅ Successfully saved artifact to: .github/workflows/ci.yml


## Lab Conclusion

Excellent! You have now generated a complete, professional Continuous Integration pipeline using AI. You created the dependency list, the containerization configuration, and the automation workflow, all from simple prompts. This is a powerful demonstration of how AI can automate complex DevOps tasks, allowing teams to build and ship software with greater speed and confidence.

> **Key Takeaway:** AI is a powerful tool for generating 'Configuration as Code' artifacts. By prompting for specific formats like `requirements.txt`, `Dockerfile`, or `ci.yml`, you can automate the creation of the files that define your entire build, test, and deployment processes.